# **KDDM 2 SARTORIUS CELL INSTANCE SEGMENTATION CELLPOSE INFERENCE NOTEBOOK**

# INSTALL All DEPENDENCIES
* **Install PyTorch version required by MMClassification**
* **Install MMClassifcation and required dependencies**
* **Install Cellpose and required dependencies**

In [ ]:
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

In [ ]:
!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/kddm2/mmcls/mmcv_full-1.3.18-cp37-cp37m-manylinux1_x86_64.whl' --no-deps

!cp -r /kaggle/input/mmdetectionv2140/pycocotools-2.0.2/* /kaggle/working/
!cp -r /kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/* /kaggle/working/

!pip install '/kaggle/working/mmpycocotools-12.0.3/' --no-deps
!pip install '/kaggle/working/pycocotools-2.0.2/' --no-deps

!rm -rf /kaggle/working/pycocotools-2.0.2/
!rm -rf /kaggle/working/mmpycocotools-12.0.3/

!rm -rf mmdetection

!cp -r /kaggle/input/kddm2/mmcls/mmclassification-0.18.0/mmclassification-0.18.0 /kaggle/working/mmclassification/
%cd /kaggle/working/mmclassification
!pip install -e .

In [ ]:
!pip install /kaggle/input/kddm2/cellpose/fastremap-1.12.2-cp37-cp37m-manylinux2010_x86_64.whl --no-deps
!pip install /kaggle/input/kddm2/cellpose/natsort-8.0.1-py3-none-any.whl --no-deps
!pip install /kaggle/input/kddm2/cellpose/pytorch_ranger-0.1.1-py3-none-any.whl --no-deps
!pip install /kaggle/input/kddm2/cellpose/torch_optimizer-0.3.0-py3-none-any.whl --no-deps
!pip install /kaggle/input/kddm2/cellpose/numpy-1.20.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl --no-deps
!pip install /kaggle/input/kddm2/cellpose/cellpose-0.7.2-py3-none-any.whl --no-deps
!pip install /kaggle/input/kddm2/cellpose/edt-2.1.1-cp37-cp37m-manylinux2014_x86_64.whl --no-deps

* **Check Installation**

In [ ]:
!nvidia-smi

import torch
print(torch.__version__, torch.cuda.is_available(),  torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'NO CUDA DEVICE')
!nvcc --version

# Check mmcv installation
import mmcv
print(mmcv.__version__)

# Check MMClassification installation
import mmcls
print(mmcls.__version__)

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print("Cuda:", get_compiling_cuda_version(), "Compiler:", get_compiler_version())

* **Enable Cellpose logging for notebooks**

In [ ]:
with open("/opt/conda/lib/python3.7/site-packages/cellpose/core.py", "r") as f:
    core_file = f.read().split("\n")
core_file[35] = "core_logger.addHandler(logging.StreamHandler(stream=sys.stdout))"
with open("/opt/conda/lib/python3.7/site-packages/cellpose/core.py", "w") as f:
    f.write( ("\n").join(core_file) )

# **INFERENCE**

* **Inference script has to be written to file due to numpy version issues on Kaggle notebooks**

In [ ]:
%%writefile run.py
from cellpose import models, io, plot
from mmcls.apis import init_model, inference_model
import numpy as np
import os
import pandas as pd
from pathlib import Path

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


classes = ["astro", "cort", "shsy5y"]
test_dir = Path('/kaggle/input/sartorius-cell-instance-segmentation/test')
test_files = [fname for fname in test_dir.iterdir()]

#set and initialize MMClassification model used to select class-specific cellpose model
mmcls_model = init_model("/kaggle/input/kddm2/mmcls/mmcls_resnext50_sartorius.py", "/kaggle/input/kddm2/mmcls/mmcls_resnext50_0.9836_sartorius.pth")
cp_model_dir = "/kaggle/input/kddm2/cellpose/"
#define which version of cellpose model should be used
cp_model_files = ["astro4", "cort9", "shsy5y5"]
#set per class parameters for cellpose models (diameter, flow threshold, mask threshold)
params = [{"dm": 25, "ft": 0.5, "mt": -0.3}, {"dm": 16, "ft": 0.3, "mt": -0.3}, {"dm": 23, "ft": 0.5, "mt": 0.3}]
#initialize the three cellpose models
cp_models = []
for i in range(3):
    cp_models.append(models.CellposeModel(gpu=True, pretrained_model=os.path.join(cp_model_dir, cp_model_files[i])))

#run inference for all test files and store predictions in competition submission format
ids, masks = [],[]
for fn in test_files:
    print(fn)
    res = inference_model(mmcls_model, str(fn))
    pred_label = int(res["pred_label"])
    print("{:s}: class {:s} ({:d}) - conf: {:.3f}".format(fn.stem, classes[pred_label], pred_label, res["pred_score"]))

    cp_model = cp_models[pred_label]
    p = params[pred_label]
    preds, flows, _ = cp_model.eval(io.imread(str(fn)), diameter=p["dm"], channels=[0,0], augment=True, resample=True, flow_threshold=p["ft"], mask_threshold=p["mt"], omni=False)
    for i in range (1, preds.max() + 1):
        ids.append(fn.stem)
        masks.append(rle_encode(preds == i))
        
pd.DataFrame({'id': ids, 'predicted': masks}).to_csv('/kaggle/working/submission.csv', index=False)
print(pd.read_csv('/kaggle/working/submission.csv').head())

* **Execute inference script**

In [ ]:
!python run.py

* **Visualize result images**

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

with open("/kaggle/working/submission.csv", "r") as f:
    anns = f.read().split("\n")
anns = anns[1:-1]
print("number of annotations", len(anns))

#convert predictions stored in csv file
masks_per_id = {}
for a in anns:
    if len(a) < 1:
        continue
    values = a.split(",")
    id = values[0]
    rle_mask = values[1].split(" ")
    starts = list(map(lambda x: int(x) - 1, rle_mask[0::2]))
    lengths = list(map(int, rle_mask[1::2]))

    ends = [x + y for x, y in zip(starts, lengths)]

    if id not in masks_per_id.keys():
        masks_per_id[id] = []

    masks_per_id[id].append( [starts, ends] )
print("number of img ids", len(masks_per_id.keys()))

#add mask overlay to images and store masked images
img_dir = '/kaggle/input/sartorius-cell-instance-segmentation/test'
for f in os.listdir(img_dir):
        if (f[-3:] != "png" and f[-3:] != "tif") or "_" in f:
            continue
            
        im = cv2.imread( "{:s}/{:s}".format(img_dir, f), cv2.IMREAD_GRAYSCALE )

        if f[:-4] not in list(masks_per_id.keys()):
            continue
        masks = masks_per_id[f[:-4]]
        
        print(f, len(masks))

        masked_img = np.empty((im.shape[0], im.shape[1], 3), dtype=np.uint8)
        masked_img[:, :, 0] = im
        masked_img[:, :, 1] = im
        masked_img[:, :, 2] = im

        flattened = masked_img.reshape( (masked_img.shape[0]*masked_img.shape[1], 3) )
        c = 0
        for m in masks:
            c += 1
            color = np.random.choice(range(256), size=3)
            for start, end in zip(m[0], m[1]):
                if start >= end:
                    print("ERROR1")
                for y in range(start, end):
                    if flattened[y, 0] == 255 and flattened[y, 1] == 0 and flattened[y, 2] == 0:
                        print("ERROR2")
                flattened[start:end] = color

        masked_img = flattened.reshape( (masked_img.shape[0], masked_img.shape[1], 3) )
        cv2.imwrite( "/kaggle/working/{:s}.png".format(f[:-4]), masked_img )
        plt.figure(figsize=(13, 13))
        plt.title(f)
        plt.imshow(masked_img)
plt.show()

# **CLEAN UP (REQUIRED FOR COMPETITION)**

In [ ]:
!ls /kaggle/working
%cd /kaggle/working
!rm -rf /kaggle/working/mmclassification
!ls /kaggle/working